In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# Swiss army knife function to organize the data

def encode(train, test):
    le = LabelEncoder().fit(train.species) 
    labels = le.transform(train.species)           # encode species strings
    classes = list(le.classes_)                    # save column names for submission
    test_ids = test.id                             # save test ids for submission
    
    train["label"] = labels
    train = train.drop(['species', 'id'], axis=1)  
    test = test.drop(['id'], axis=1)
    
    return train, labels, test, test_ids, classes

train, labels, test, test_ids, classes = encode(train, test)
train.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64,label
0,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,0.033203,...,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391,3
1,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,0.007812,...,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461,49
2,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,0.044922,...,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930,65
3,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,0.017578,...,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852,94
4,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,0.005859,...,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250,84


In [4]:
sss = StratifiedShuffleSplit(labels, 10, test_size=0.2, random_state=23)

for train_index, test_index in sss:
    X_train, X_test = train.loc[train_index], train.loc[test_index]

In [5]:
X_train.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64,label
638,0.000000,0.000000,0.074219,0.044922,0.003906,0.000000,0.000000,0.0,0.013672,0.000000,...,0.001953,0.018555,0.000977,0.007812,0.000000,0.004883,0.051758,0.000000,0.000977,59
797,0.017578,0.009766,0.056641,0.029297,0.007812,0.017578,0.037109,0.0,0.005859,0.062500,...,0.000000,0.015625,0.017578,0.026367,0.001953,0.000000,0.022461,0.000000,0.002930,81
600,0.001953,0.000000,0.003906,0.007812,0.017578,0.000000,0.025391,0.0,0.000000,0.009766,...,0.001953,0.011719,0.006836,0.022461,0.000000,0.000000,0.025391,0.007812,0.048828,12
908,0.037109,0.017578,0.031250,0.023438,0.001953,0.078125,0.039062,0.0,0.003906,0.042969,...,0.007812,0.000000,0.155270,0.005859,0.087891,0.000000,0.076172,0.000977,0.000000,27
434,0.000000,0.000000,0.009766,0.021484,0.058594,0.000000,0.009766,0.0,0.011719,0.000000,...,0.049805,0.000977,0.005859,0.008789,0.000000,0.000000,0.041016,0.000000,0.009766,8


In [6]:
FEATURES = test.columns
FEATURES

Index([u'margin1', u'margin2', u'margin3', u'margin4', u'margin5', u'margin6',
       u'margin7', u'margin8', u'margin9', u'margin10',
       ...
       u'texture55', u'texture56', u'texture57', u'texture58', u'texture59',
       u'texture60', u'texture61', u'texture62', u'texture63', u'texture64'],
      dtype='object', length=192)

In [7]:
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]

In [8]:
#?tf.contrib.learn.DNNClassifier()

In [10]:
dnn_classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_cols, hidden_units=[512, 256],n_classes=99)

linear_classifier = tf.contrib.learn.LinearClassifier(feature_columns=feature_cols,n_classes=99)

In [11]:
def input_fn(df):
    feature_cols = {k: tf.constant(df[k].values) for k in FEATURES}
    label = tf.constant(df["label"].values)
    return feature_cols, label

In [13]:
dnn_classifier.fit(input_fn=lambda: input_fn(X_train), steps=2000)

DNNClassifier(hidden_units=[512, 256], dropout=None, optimizer=None, feature_columns=[_RealValuedColumn(column_name='margin1', dimension=1, default_value=None, dtype=tf.float32), _RealValuedColumn(column_name='margin2', dimension=1, default_value=None, dtype=tf.float32), _RealValuedColumn(column_name='margin3', dimension=1, default_value=None, dtype=tf.float32), _RealValuedColumn(column_name='margin4', dimension=1, default_value=None, dtype=tf.float32), _RealValuedColumn(column_name='margin5', dimension=1, default_value=None, dtype=tf.float32), _RealValuedColumn(column_name='margin6', dimension=1, default_value=None, dtype=tf.float32), _RealValuedColumn(column_name='margin7', dimension=1, default_value=None, dtype=tf.float32), _RealValuedColumn(column_name='margin8', dimension=1, default_value=None, dtype=tf.float32), _RealValuedColumn(column_name='margin9', dimension=1, default_value=None, dtype=tf.float32), _RealValuedColumn(column_name='margin10', dimension=1, default_value=None, dt

In [14]:
dnn_results = dnn_classifier.evaluate(input_fn=lambda: input_fn(X_test), steps=1)

In [15]:
for key in sorted(dnn_results):
    print "%s: %s" %(key, dnn_results[key])

accuracy: 0.843434
global_step: 2000
loss: 0.817941


In [16]:
linear_classifier.fit(input_fn=lambda: input_fn(X_train), steps=10000)

LinearClassifier()

In [ ]:
lin_results = linear_classifier.evaluate(input_fn=lambda:input_fn(X_test),steps=1)

In [ ]:
for key in sorted(lin_results):
    print "%s: %s" %(key, lin_results[key])